# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile
import os

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# Get the current working directory
current_dir = os.getcwd()

# Path to the Orders.zip file 
zip_file_path = os.path.join(current_dir, 'Orders.zip')

In [5]:
# Set the extraction path 
extraction_path = os.path.join(current_dir, 'extracted_files')
os.makedirs(extraction_path, exist_ok=True)  # Create the extraction directory 

In [6]:
# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [10]:
orders = pd.read_csv(f"{extraction_path}/Orders.csv")

In [11]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [15]:
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [16]:
## Sub Problem 1: How to aggregate the amount_spent for unique customers?

# Group by 'CustomerID' and sum the 'amount_spent'
customer_spending = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()

# Rename the columns for better understanding
customer_spending.columns = ['CustomerID', 'total_spent']

# Display the aggregated customer spending
customer_spending.head()


,CustomerID,total_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [19]:
## Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

# Calculate the 95th and 75th percentiles for total spending
vip_threshold = customer_spending['total_spent'].quantile(0.95)
preferred_threshold = customer_spending['total_spent'].quantile(0.75)

# Filter customers
vip_customers = customer_spending[customer_spending['total_spent'] > vip_threshold]
preferred_customers = customer_spending[(customer_spending['total_spent'] <= vip_threshold) & (customer_spending['total_spent'] > preferred_threshold)]

# Display
print(f"VIP customers:\n{vip_customers}")
print(f"Preferred customers:\n{preferred_customers}")


VIP customers:
      CustomerID  total_spent
0          12346     77183.60
10         12357      6207.67
12         12359      6372.58
50         12409     11072.67
55         12415    124914.53
...          ...          ...
4207       18109      8052.97
4229       18139      8438.34
4253       18172      7561.68
4292       18223      6484.54
4298       18229      7276.90

[217 rows x 2 columns]
Preferred customers:
      CustomerID  total_spent
1          12347      4310.00
2          12348      1797.24
3          12349      1757.55
5          12352      2506.04
9          12356      2811.43
...          ...          ...
4319       18259      2338.60
4320       18260      2643.20
4328       18272      3078.58
4337       18283      2094.88
4338       18287      1837.28

[868 rows x 2 columns]


In [25]:
## Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

# Add a new column to label customers
customer_spending['label'] = 'Regular' # Set to default label

# Label VIP customers
customer_spending.loc[customer_spending['total_spent'] > vip_threshold, 'label'] = 'VIP'

# Label Preferred customers
customer_spending.loc[(customer_spending['total_spent'] <= vip_threshold) & (customer_spending['total_spent'] > preferred_threshold), 'label'] = 'Preferred'

customer_spending.head()


,CustomerID,total_spent,label
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [33]:
# Merge customer spending with the original orders df to get country information
label_customers = pd.merge(orders[['CustomerID', 'Country']], customer_spending[['CustomerID', 'label']], on='CustomerID', how='outer')

# Filter for VIP customers
vip_customers = label_customers[label_customers['label'] == 'VIP']

# Group by country and count the number of VIP customers in each country
vip_country = vip_customers.groupby('Country')['CustomerID'].count().reset_index()

# Rename columns
vip_country.columns = ['Country','VIP_count']

# Sort by 'VIP_count' to identify the country with the most VIP customers
vip_country = vip_country.sort_values(by='VIP_count', ascending=False)

# Display
top_vip_country = vip_country.head(10)
print(f"Top 10 countries with most VIP customers:\n{top_vip_country}")




Top 10 countries with most VIP customers:
           Country  VIP_count
17  United Kingdom      84185
5             EIRE       7077
7           France       3290
8          Germany       3127
10     Netherlands       2080
0        Australia        898
12        Portugal        681
16     Switzerland        594
14           Spain        511
11          Norway        420


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [34]:
# Filter for VIP or Preferred customers
vip_preferred_customers = label_customers[label_customers['label'].isin(['VIP', 'Preferred'])]

# Group by country and count the number of VIP + preferred customers in each country
vip_preferred_country = vip_preferred_customers.groupby('Country')['CustomerID'].count().reset_index()

# Rename columns
vip_preferred_country.columns = ['Country','VIP_and_Preferred_count']

# Sort by 'VIP_and_Preferred_count' to identify the country with the most VIP + preferred customers
vip_preferred_country = vip_preferred_country.sort_values(by='VIP_and_Preferred_count', ascending=False)

# Display
top_vip_preferred_country = vip_preferred_country.head(10)
print(f"Top countries with the most VIP and preferred customers combined:\n{top_vip_preferred_country}")


Top countries with the most VIP and preferred customers combined:
           Country  VIP_and_Preferred_count
26  United Kingdom                   221635
10         Germany                     7349
7             EIRE                     7238
9           France                     6301
18     Netherlands                     2080
23           Spain                     1569
2          Belgium                     1557
25     Switzerland                     1370
21        Portugal                     1093
19          Norway                     1028
